In [1]:
import pandas as pd
import xml.etree.ElementTree as ET
import psycopg2
from datetime import datetime
import re
import cx_Oracle
import numpy as np
# Database connection parameters
db_params = {
    "host": "gc-ue4-psql-cspo-dev01.nonprod.gcp.cswg.com",
    "database": "CSPODB",
    "port": 5432,
    "user": "cspoms",
    "password": "cspoms",
    "sslmode": "verify-ca",
    "sslcert": "D:\\W\\RELEX\\RELEX\\CSPOMS_DEV_Conn\\client-cert.pem",
    "sslkey": "D:\\W\\RELEX\\RELEX\\CSPOMS_DEV_Conn\\client-key.pem",
    "sslrootcert": "D:\\W\\RELEX\\RELEX\\CSPOMS_DEV_Conn\\server-ca.pem"
}

post_conn = psycopg2.connect(**db_params)
post_cur = post_conn.cursor()

In [2]:
import pandas as pd

post_cur.execute("""SELECT *
FROM XXPO_PULLBAK_PARAM
WHERE LPAD(CAST(to_fac_code AS VARCHAR), 3, '0') IN (
    SELECT LPAD(CAST(facility_code AS VARCHAR), 3, '0')
    FROM cspoms_replenishment_config
    WHERE POMS_SOURCE = 'CSPOMS'
    AND STATUS = 'Y'
)""")
data = post_cur.fetchall()

# Get column names from the cursor description
column_names = [desc[0] for desc in post_cur.description]

# Create DataFrame with data and column names
df = pd.DataFrame(data, columns=column_names)
df = df[['from_fac_code','from_warehouse_num','to_fac_code','incr','vendor_number','max_pallets_allowed','max_weight_allowed','max_cube_allowed','lead_time','op_days']]
# Convert columns to integer type
columns_to_convert = ['from_fac_code','from_warehouse_num','to_fac_code']
# Replace or drop non-finite values
df[columns_to_convert] = df[columns_to_convert].replace([np.inf, -np.inf], np.nan).fillna(0)  # Replace inf with NaN and fill NaN with 0

# Convert columns to integer type
df[columns_to_convert] = df[columns_to_convert].astype(int)
# Dictionary with new column names
new_column_names = {'from_fac_code': "T2_TABLE_26_FR_FAC",
                    'from_warehouse_num': "T2_TABLE_26_FR_WH",
                    'to_fac_code': "T2_TABLE_26_TO_FAC",
                    'incr': "T2_TABLE_26_ORD_INCR",
                    'vendor_number': "T2_TABLE_26_VENDOR",
                    'max_pallets_allowed': "T2_TABLE_26_MIN_PALLET",
                    'max_weight_allowed': "T2_TABLE_26_MIN_WGHT",
                    'max_cube_allowed': "T2_TABLE_26_MIN_CUBE",
                    'lead_time': "T2_TABLE_26_LEAD_TIME",
                    'op_days': "T2_TABLE_26_OP_DAYS"}
# Original column names with their specified lengths
original_column_names = {
    "FROM_FAC_CODE": 13,
    "FROM_WAREHOUSE_NUM": 18,
    "TO_FAC_CODE": 13,
    "INCR": 8,
    "VENDOR_NUMBER": 14,
    "MAX_PALLETS_ALLOWED": 20,
    "MAX_WEIGHT_ALLOWED": 18,
    "MAX_CUBE_ALLOWED": 16,
    "LEAD_TIME": 10,
    "OP_DAYS": 10
}

# Adjust column names to the specified lengths
adjusted_column_names = {col: col.ljust(length) for col, length in original_column_names.items()}

# Use the rename method to rename columns
df.rename(columns=adjusted_column_names, inplace=True)

# Display the DataFrame with adjusted column names
print(df)
# Use the rename method to rename columns
df.rename(columns=new_column_names, inplace=True)

# Assuming df is your DataFrame
# df = pd.concat([df.iloc[:, :10], df.iloc[:, :10]], axis=1)

# Printing the first few rows of the duplicated DataFrame
print(df.head())
# Reorder columns to have first 3 columns at the beginning
# Copy the first three columns to the beginning of the DataFrame
# df = pd.concat([df[['T2_TABLE_26_FR_FAC', 'T2_TABLE_26_FR_WH', 'T2_TABLE_26_TO_FAC']], df], axis=1)
# Assuming df is your DataFrame
# df = df.astype(str)

df['LAST_UPDATED_BY'] = -999
# # Assuming df is your DataFrame
df = pd.concat([df.iloc[:, :11], df.iloc[:, :11]], axis=1)

# # Printing the first few rows of the duplicated DataFrame
# print(df.head())
df

   from_fac_code  from_warehouse_num  to_fac_code incr vendor_number  \
0             15                   2           23    P        777965   
1             15                   2           43    P        777965   

   max_pallets_allowed  max_weight_allowed  max_cube_allowed  lead_time  \
0                   60               42000              1500          1   
1                   60               42000              1500          1   

   op_days  
0        3  
1        3  
   T2_TABLE_26_FR_FAC  T2_TABLE_26_FR_WH  T2_TABLE_26_TO_FAC  \
0                  15                  2                  23   
1                  15                  2                  43   

  T2_TABLE_26_ORD_INCR T2_TABLE_26_VENDOR  T2_TABLE_26_MIN_PALLET  \
0                    P             777965                      60   
1                    P             777965                      60   

   T2_TABLE_26_MIN_WGHT  T2_TABLE_26_MIN_CUBE  T2_TABLE_26_LEAD_TIME  \
0                 42000                  1500

,T2_TABLE_26_FR_FAC,T2_TABLE_26_FR_WH,T2_TABLE_26_TO_FAC,T2_TABLE_26_ORD_INCR,T2_TABLE_26_VENDOR,T2_TABLE_26_MIN_PALLET,T2_TABLE_26_MIN_WGHT,T2_TABLE_26_MIN_CUBE,T2_TABLE_26_LEAD_TIME,T2_TABLE_26_OP_DAYS,...,T2_TABLE_26_FR_WH,T2_TABLE_26_TO_FAC,T2_TABLE_26_ORD_INCR,T2_TABLE_26_VENDOR,T2_TABLE_26_MIN_PALLET,T2_TABLE_26_MIN_WGHT,T2_TABLE_26_MIN_CUBE,T2_TABLE_26_LEAD_TIME,T2_TABLE_26_OP_DAYS,LAST_UPDATED_BY
0,15,2,23,P,777965,60,42000,1500,1,3,...,2,23,P,777965,60,42000,1500,1,3,-999
1,15,2,43,P,777965,60,42000,1500,1,3,...,2,43,P,777965,60,42000,1500,1,3,-999


In [ ]:

# Broker_vendor.py
import pandas as pd
import xml.etree.ElementTree as ET
import psycopg2
from datetime import datetime
import re
import cx_Oracle

username = 'ebs_apps'
password = 'ebs_apps'
host = 'csdwddb.cswg.com' #EBSD2
port = '1521'
service_name = 'DWD_USR' #EBSD2

# Create a connection
dsn = cx_Oracle.makedsn(host, port, service_name=service_name)
oracle_con = cx_Oracle.connect(username, password, dsn)
oracle_cur = oracle_con.cursor()

# dfEXA = oracle_cur.execute('select * from XXPO_PULLBACK_BICEPS2EXA_STG')
# dfEXA.fetchall()

# Execute the merge statement
merge_sql = """

MERGE INTO XXPO_PULLBACK_BICEPS2EXA tgt
USING (SELECT * FROM dual) src
ON (tgt.T2_TABLE_26_FR_FAC = :1 AND tgt.T2_TABLE_26_FR_WH = :2 AND tgt.T2_TABLE_26_TO_FAC = :3)
WHEN NOT MATCHED THEN

INSERT (T2_TABLE_26_FR_FAC, T2_TABLE_26_FR_WH, T2_TABLE_26_TO_FAC, T2_TABLE_26_ORD_INCR,
          T2_TABLE_26_VENDOR, T2_TABLE_26_MIN_PALLET, T2_TABLE_26_MIN_WGHT, T2_TABLE_26_MIN_CUBE,
          T2_TABLE_26_LEAD_TIME, T2_TABLE_26_OP_DAYS, LAST_UPDATED_BY)
VALUES (:4, :5, :6, :7, :8, :9, :10, :11, :12, :13, :14)
WHEN MATCHED THEN
UPDATE SET
    T2_TABLE_26_ORD_INCR = :15,
    T2_TABLE_26_VENDOR = :16,
    T2_TABLE_26_MIN_PALLET = :17,
    T2_TABLE_26_MIN_WGHT = :18,
    T2_TABLE_26_MIN_CUBE = :19,
    T2_TABLE_26_LEAD_TIME = :20,
    T2_TABLE_26_OP_DAYS = :21,
    LAST_UPDATED_BY = :22
"""

# Convert DataFrame to list of tuples
data = [tuple(row) for row in df.values]

# Execute the merge statement for each row of data
for row in data:
    # Print the values in the row tuple
    print("Row tuple:", row)
    oracle_cur.execute(merge_sql,row)
    


# Commit the transaction and close the cursor and connection
oracle_con.commit()
oracle_cur.close()
oracle_con.close()


Row tuple: (15, 2, 23, 'P', '777965', 60, 42000, 1500, 1, 3, -999, 15, 2, 23, 'P', '777965', 60, 42000, 1500, 1, 3, -999)


In [10]:
new_column_names_swapped = {v: f'"{k}"' for k, v in new_column_names.items()}

# Print the resulting dictionary
print(new_column_names_swapped)


{'"T2-TABLE-26-FR-FAC"': '"from_fac_code"', 'from_fac_code': '"T2-TABLE-26-FR-FAC"', 'from_warehouse_num': '"T2-TABLE-26-FR-WH"', 'to_fac_code': '"T2-TABLE-26-TO-FAC"', 'incr': '"T2-TABLE-26-ORD-INCR"', 'vendor_number': '"T2-TABLE-26-VENDOR"', 'max_pallets_allowed': '"T2-TABLE-26-MIN-PALLET"', 'max_weight_allowed': '"T2-TABLE-26-MIN-WGHT"', 'max_cube_allowed': '"T2-TABLE-26-MIN-CUBE"', 'lead_time': '"T2-TABLE-26-LEAD-TIME"', 'op_days': '"T2-TABLE-26-OP-DAYS"'}


In [4]:
import pandas as pd

post_cur.execute("SELECT * FROM XXPO_PULLBAK_PARAM")
data = post_cur.fetchall()
data.columns
column_names = [import csv

# Specify the custom delimiter as |
custom_delimiter = '|'
file_name = 'CSPOMS_EXA_I1753_RLXPULL.date.time.txt'
csv_filepath_push = 'D:\\W\\RELEX\\RELEX\\I1674 - Po recon\\CSPOMS_EXA_I1753_RLXPULL.date.time.csv'

# Export the DataFrame to CSV with custom delimiter, column names in the first row, and column names enclosed in double quotes
df.to_csv(csv_filepath_push, sep=custom_delimiter, header=True, index=False, quoting=csv.QUOTE_NONNUMERIC)
]

df = pd.DataFrame(data, columns=column_names)
print(df)


AttributeError: 'list' object has no attribute 'columns'

In [27]:
# Specify the custom delimiter as |
custom_delimiter = '|'
file_name = f'CSPOMS_EXA_I1753_RLXPULL.date.time.txt'
csv_filepath_push = 'D:\\W\\RELEX\\RELEX\\I1674 - Po recon\\CSPOMS_EXA_I1753_RLXPULL.date.time.csv'
# D:\W\RELEX\RELEX\I1674 - Po recon\relex_I1674_po_recon_dag.py
# Export the DataFrame to CSV with custom delimiter
df.to_csv(csv_filepath_push, sep=custom_delimiter, header=True, index=False)

In [28]:
import csv

# Specify the custom delimiter as |
custom_delimiter = '|'
file_name = 'CSPOMS_EXA_I1753_RLXPULL.date.time.txt'
csv_filepath_push = 'D:\\W\\RELEX\\RELEX\\I1674 - Po recon\\CSPOMS_EXA_I1753_RLXPULL.date.time.csv'

# Export the DataFrame to CSV with custom delimiter, column names in the first row, and column names enclosed in double quotes
df.to_csv(csv_filepath_push, sep=custom_delimiter, header=True, index=False, quoting=csv.QUOTE_NONNUMERIC)
